<a href="https://colab.research.google.com/github/Leo-BM/peyton-manning-forecasting/blob/main/peyton_manning_wiki_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Estudo de caso do número de visitantes diários na página da Wikipedia do ex-jogador da NFL Peyton Manning

## 1. Configuração do Ambiente e Ferramentas

In [ ]:
!pip install neuralprophet

In [ ]:
#pandas
#ploty
#sklearn
#prophet
#numpy
#matplotlib pyplot e seaborn
#statsmodels para decomposição da série

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from prophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

%matplotlib inline
sns.set(style='whitegrid')


##2. Análise Exploratória dos Dados (EDA)

###Limpeza e formatação

In [ ]:
#Carregue sua base de dados aqui
dataset = pd.read_csv('page_wikipedia.csv')

In [ ]:
dataset.head()

In [ ]:
dataset.tail()

Data do primeiro registro: 10/12/2007
Data do último registro: 20/01/2016

In [ ]:
dataset.shape

In [ ]:
dataset.describe()

#### OBS: Já foi aplicado o logaritmo na coluna "views" da série. Isso é realizado para a estabilização da variância dos dados. Como veremos adiante, esses dados possuem dias em que há uma alta significativa no número de visualizações. Para acentuar essa variância e modelar a série de uma melhor maneira opta-se por trabalhar com o logaritimo.

In [ ]:
#Verificação dos tipos de objeto do dataset

dataset.dtypes

In [ ]:
#Renomear as colunas para o padrão de uso  da biblioteca Prophet

dataset = dataset[['date', 'views']].rename(columns = {'date': 'ds', 'views': 'y'})

In [ ]:
#Transformação da coluna "ds" em um objeto datetime

dataset['ds'] = pd.to_datetime(dataset['ds'])

In [ ]:
dataset

In [ ]:
mean_views = dataset['y'].mean()
median_views = dataset['y'].median()

dataset['y'].hist()
plt.axvline(mean_views, color='red', linestyle='dashed', linewidth=2, label=f'Média: {mean_views:.2f}')
plt.axvline(median_views, color='green', linestyle='dashed', linewidth=2, label=f'Mediana: {median_views:.2f}')
plt.title('Distribuição de frequência do número de visualizações diárias da página do atleta')
plt.xlabel('Número de Visualizações (log)')
plt.ylabel('Frequência')
plt.legend()
plt.show()

### Visualização da Série com a biblioteca Ploty

In [ ]:
fig = px.line(dataset, x = 'ds', y = 'y')
fig.show()
#superbowls jogados pelo atleta -> 2007, 2010, 2014, 2016

In [ ]:
#média móvel de 30 períodos da série
dataset_ma = dataset['y'].rolling(window = 30).mean()

In [ ]:
database = dataset.copy()

In [ ]:
database['y_ma'] = dataset_ma

In [ ]:
fig = px.line(database, x='ds', y=['y', 'y_ma'], title= 'Views com média móvel de 30 períodos')
fig.show()

In [ ]:
#Boxplot para uma análise sazonal da série

dataset['month'] = dataset['ds'].dt.month
dataset['day_of_week'] = dataset['ds'].dt.day_name()

In [ ]:
dataset.head()

In [ ]:
plt.figure(figsize=(15,6))
sns.boxplot(data = dataset, x = 'month', y = 'y', palette='viridis')
plt.title('Distribuição de Acessos por Mês (Peyton Manning)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
# Definindo a ordem para começar na segunda-feira
ordem_dias = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

sns.boxplot(x='day_of_week', y='y', data=dataset, order=ordem_dias, palette='Set2')
plt.title('Distribuição de Acessos por Dia da Semana')
plt.xticks(rotation=45)
plt.show()

###Decomposição Sazonal da Série

In [ ]:
##Dado que a série está em escala logarítmica, utilizaremos o modelo Aditivo, pois o log estabiliza a variância multiplicativa.
result = seasonal_decompose(dataset['y'], model='additive', period=365)

# Plotando de forma organizada
plt.rcParams['figure.figsize'] = (12, 10)
result.plot()
plt.show()

##3. Verificação Estatística

#### Implementação do teste de Dickey-Fuller (ADF) para verificação de estacionariedade


In [ ]:
result_adf = adfuller(dataset['y'])
print(result_adf[1])

Pelo ADF percebemos que a série é estacionária

####Gráficos ACF e PACF para entendimento da correlação de dias anteriores.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
plot_acf(dataset['y'], lags=50, ax=ax1)
plot_pacf(dataset['y'], lags=50, ax=ax2)
plt.show()


- ACF com decaimento lento (possui memória longa)
- PACF significativo até aproximadamente o lag = 7. Restante dos atrasos não acrescenta muita informação.


##4. Modelagem com Prophet

In [ ]:
#padronizando novamente para uso no Prophet
dataset.drop(columns=['month', 'day_of_week'])

In [ ]:
#Treinamento da série usando a biblioteca
m = Prophet()

In [ ]:
m.fit(dataset)

In [ ]:
future = m.make_future_dataframe(periods=365)

In [ ]:
forecast = m.predict(future)

In [ ]:
forecast.tail()

In [ ]:
fig_1 = m.plot(forecast)

In [ ]:
fig_2 = m.plot_components(forecast)

### Incorporação de Eventos Especiais (Holidays)

In [ ]:
#Vamos adicionar alguma datas de picos de visualizações na modelagem de nossa série
#Procuramos graficamente quais são esses "dias especiais"

feriados = pd.DataFrame({
    'holiday' : 'superbowl',
    'ds' : pd.to_datetime(['2010-02-07', '2014-02-03', '2016-02-07']),
    'lower_window' : 0,
    'upper_window' : 1,
})

In [ ]:
feriados

In [ ]:
#Instanciar o modelo com o nosso parâmetro criado

m = Prophet(holidays = feriados)
m.fit(dataset)
forecast_2 = m.predict(future)

In [ ]:
fig_3 = m.plot_components(forecast_2)

In [ ]:
fig_4 = m.plot(forecast)

##5. Avaliação de Performance


###Validação cruzada no Prophet

Período de 180 registros entre um teste e outro, com 730 dias de base para o ínicio da validação, com previsão de intervalo de 365 dias

In [ ]:
from prophet.diagnostics import cross_validation

ds_cross_valid = cross_validation(m, initial='730 days', period='180 days', horizon = '365 days')


In [ ]:
## Vamos analisar algumas métricas de erro da nossa validação cruzada
from prophet.diagnostics import performance_metrics
ds_pf = performance_metrics(ds_cross_valid)
ds_pf.head()

In [ ]:
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(ds_cross_valid, metric='mape')


##Comparativo entre modelos Prophet, SARIMAX, e Neural Prophet

Vamos treinar os modelos com validação cruzada e fazer uma avaliação de desempenho.

In [ ]:
dataset.head()

In [ ]:
dataset = dataset.drop(columns=['month', 'day_of_week'])

In [ ]:
print(len(dataset))

In [ ]:
 #Criação de uma base de treinamento excluindo 365 dias da base original

train_size = len(dataset) - 365
print(train_size)

In [ ]:
dataset_train = dataset.iloc[:train_size]
print(len(dataset_train))

In [ ]:
dataset_test = dataset.iloc[train_size:]
print(len(dataset_test))

In [ ]:
!pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima

In [ ]:
#Treinando um modelo ARIMA com auto_arima

dataset_arima = auto_arima(dataset_train['y'],
                           m=7,
                           seasonal=True,
                           trace=True,
                           error_action='ignore',
                           suppress_warnings=True)

In [ ]:
dataset_arima.summary()

In [ ]:
sarimax_predict = dataset_arima.predict(n_periods=len(dataset_test))

In [ ]:
sarimax_predict

In [ ]:
from neuralprophet import NeuralProphet

# Instanciando
# Ele também aceita holidays, mas vamos testar o modelo puro primeiro para ver o poder da rede neural
m_neural = NeuralProphet(learning_rate=0.01)
# Treinando
metrics = m_neural.fit(dataset_train, freq="D")
if not hasattr(np, 'NaN'):
    np.NaN = np.nan
# Prevendo
# O NeuralProphet precisa de um dataframe futuro específico
future_neural = m_neural.make_future_dataframe(dataset_train, periods=len(dataset_test))
forecast_neural = m_neural.predict(future_neural)

# Extraindo apenas a predição final (yhat1)
preds_neural = forecast_neural['yhat1'].tail(len(dataset_test)).values

In [ ]:
preds_neural

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# 1. Recalcular Prophet para essa janela de treino e teste
m_prophet = Prophet(holidays=feriados)
m_prophet.fit(dataset_train)
future_prophet = m_prophet.make_future_dataframe(periods=len(dataset_test))
forecast_prophet = m_prophet.predict(future_prophet)
preds_prophet = forecast_prophet['yhat'].tail(len(dataset_test)).values

# Dataframe para comparação
comparativo = pd.DataFrame({
    'Data': dataset_test['ds'],
    'Real': dataset_test['y'],
    'Prophet': preds_prophet,
    'SARIMAX': sarimax_predict,
    'NeuralProphet': preds_neural
})

# Métricas de erro
print("--- Batalha dos Modelos (RMSE) ---")
print(f"Prophet: {mean_squared_error(comparativo['Real'], comparativo['Prophet']):.4f}")
print(f"SARIMAX: {mean_squared_error(comparativo['Real'], comparativo['SARIMAX']):.4f}")
print(f"NeuralProphet: {mean_squared_error(comparativo['Real'], comparativo['NeuralProphet']):.4f}")

# Erros percentuais
print("--- Batalha dos Modelos Percentual ---")
print(f"Prophet percentual: {mean_absolute_percentage_error(comparativo['Real'], comparativo['Prophet']):.4f}")
print(f"SARIMAX percentual: {mean_absolute_percentage_error(comparativo['Real'], comparativo['SARIMAX']):.4f}")
print(f"NeuralProphet percentual: {mean_absolute_percentage_error(comparativo['Real'], comparativo['NeuralProphet']):.4f}")

# 4. Plotagem
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=comparativo['Data'], y=comparativo['Real'], name='Real', line=dict(color='black')))
fig.add_trace(go.Scatter(x=comparativo['Data'], y=comparativo['Prophet'], name='Prophet', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=comparativo['Data'], y=comparativo['SARIMAX'], name='SARIMAX', line=dict(color='green')))
fig.add_trace(go.Scatter(x=comparativo['Data'], y=comparativo['NeuralProphet'], name='NeuralProphet', line=dict(color='red')))
fig.show()